In [17]:
import os

In [18]:
os.environ["MLFLOW_TRACKING_URI"] = 'https://dagshub.com/ArpitKadam/data-science-project.mlflow'
os.environ["MLFLOW_TRACKING_USERNAME"] = 'ArpitKadam'
os.environ["MLFLOW_TRACKING_PASSWORD"] = '5989d6b56c4eec6ea090d927851d1fb5297a42a8'

In [4]:
os.chdir('../')

In [5]:
pwd

'c:\\Users\\Arpit Kadam\\Desktop\\DataScienceProject\\data-science-project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from src.datascienceproject.constants import *
from src.datascienceproject.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH, 
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        # Create root directory
        create_directories([self.config["artifacts"]["artifacts_root"]])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/ArpitKadam/data-science-project.mlflow"
            )

        return model_evaluation_config

In [9]:
import os
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
from urllib.parse import urlparse
import joblib 

In [15]:
class ModelEvaluation: 
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        mse = mean_squared_error(actual, pred)
        rmse = root_mean_squared_error(actual, pred)
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return mse, rmse, mae, r2

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        x_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_registry_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(x_test)
            
            (mse, rmse, mae, r2) = self.eval_metrics(y_test, predicted_qualities)

            scores = {
                "mse": mse,
                "rmse": rmse,
                "mae": mae,
                "r2": r2
            }

            save_json(path = Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "mse": mse,
                "rmse": rmse,
                "mae": mae,
                "r2": r2
            })
           
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel") 
            else:
                mlflow.sklearn.log_model(model, "model")



In [16]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config = model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-12-26 22:09:21,393: INFO: common: Yaml file: config\config.yaml read successfully]
[2024-12-26 22:09:21,397: INFO: common: Yaml file: params.yaml read successfully]
[2024-12-26 22:09:21,402: INFO: common: Yaml file: schema.yaml read successfully]
[2024-12-26 22:09:21,405: INFO: common: Created directory at: ./artifacts]
[2024-12-26 22:09:21,407: INFO: common: Created directory at: ./artifacts/model_evaluation]
[2024-12-26 22:09:21,918: INFO: common: Json file saved at: artifacts\model_evaluation\metrics.json]


2024/12/26 22:09:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticNetModel' already exists. Creating a new version of this model...
2024/12/26 22:09:37 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 4
Created version '4' of model 'ElasticNetModel'.


🏃 View run illustrious-bird-331 at: https://dagshub.com/ArpitKadam/data-science-project.mlflow/#/experiments/0/runs/3188cbdb90014a788f16728b87f3486a
🧪 View experiment at: https://dagshub.com/ArpitKadam/data-science-project.mlflow/#/experiments/0
